<a href="https://colab.research.google.com/github/leehow-data/Lee_DTSC3020_Fall2025/blob/main/cgh0117_Assignment5_Ch_10%2611.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment5: CRM Cleanup @ **DalaShop**
*Files (Ch.10), Exceptions (Ch.10), Unit Tests (Ch.11), and Regular Expressions*  
.....

**Total: 3 points**  (Two questions, 1.5 pts each)  

> This assignment is scenario-based and aligned with Python Crash Course Ch.10 (files & exceptions), Ch.11 (unit testing with `unittest`), and Regular Expressions.

## Scenario
You are a data intern at an online retailer called **DalaShop**.  
Sales exported a **raw contacts** file from the CRM. It contains customer names, emails, and phone numbers, but the formatting is messy and some emails are invalid.  
Your tasks:

1. **Clean** the contacts (Files + Exceptions + Regex).  
2. **Write unit tests** to make sure your helper functions work correctly and keep working in the future.

## Data file (given by the company): `contacts_raw.txt`
Use this exact sample data (you may extend it for your own testing, but do **not** change it when submitting).  
Run the next cell once to create the file beside your notebook.

In [25]:
# Create the provided company dataset file
with open("contacts_raw.txt", "w", encoding="utf-8") as f:
    f.write('Alice Johnson <alice@example.com> , +1 (469) 555-1234\nBob Roberts <bob[at]example.com> , 972-555-777\nSara M. , sara@mail.co , 214 555 8888\n"Mehdi A." <mehdi.ay@example.org> , (469)555-9999\nDelaram <delaram@example.io>, +1-972-777-2121\nNima <NIMA@example.io> , 972.777.2121\nduplicate <Alice@Example.com> , 469 555 1234')
print("Wrote contacts_raw.txt with sample DalaShop data.")

Wrote contacts_raw.txt with sample DalaShop data.


## Q1 (1.5 pts) — CRM cleanup with Files, Exceptions, and Regex
Implement `q1_crm_cleanup.py` to:

1. **Read** `contacts_raw.txt` using `pathlib` and `with`. If the file is missing, **handle** it gracefully with `try/except FileNotFoundError` (print a friendly message; do not crash).
2. **Validate emails** with a simple regex (`r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"`).  
   - Trim whitespace with `strip()` before checking.  
   - Use **full** matching (not partial).
3. **Normalize phone numbers:** remove all non-digits (e.g., with `re.sub(r"\D", "", raw)`).  
   - If the result has **≥ 10 digits**, keep the **last 10 digits**.  
   - Otherwise, return an **empty string** (`""`).
4. **Filter rows:** keep **only** rows with a valid email.
5. **Deduplicate:** remove duplicates by **email** using **case-insensitive** comparison (e.g., `email.casefold()`). **Keep the first occurrence** and drop later duplicates.
6. **Output CSV:** write to `contacts_clean.csv` with **columns exactly** `name,email,phone` (UTF-8).  
7. **Preserve input order:** the order of rows in `contacts_clean.csv` must match the **first appearance** order from the input file. **Do not sort** the rows.

**Grading rubric (1.5 pts):**
- (0.4) File read/write via `pathlib` + graceful `FileNotFoundError` handling  
- (0.5) Correct email regex validation + filtering  
- (0.4) Phone normalization + case-insensitive de-dup (keep first)  
- (0.2) Clean code, clear names, minimal docstrings/comments

In [31]:
# Write your answer here
import re
import csv
from pathlib import Path


def is_valid_email(email):
    """Return True if email fully matches a valid email regex."""
    pattern = re.compile(r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$")
    email = email.strip()
    return bool(pattern.fullmatch(email))


def normalize_phone(raw):
    """Remove non-digits and keep last 10 digits if available."""
    digits = re.sub(r"\D", "", raw)
    if len(digits) >= 10:
        return digits[-10:]
    return ""


def clean_contacts(input_path="contacts_raw.txt", output_path="contacts_clean.csv"):
    """Clean contacts file and write cleaned data to CSV."""
    input_file = Path(input_path)
    output_file = Path(output_path)

    try:
        lines = input_file.read_text(encoding="utf-8").splitlines()
    except FileNotFoundError:
        print(f"File not found: {input_file.name}. Please make sure it exists.")
        return

    cleaned_rows = []
    seen_emails = set()

    for line in lines:
        parts = [p.strip() for p in re.split(r"[<>,]", line) if p.strip()]
        name, email, phone = "", "", ""
        for p in parts:
            if is_valid_email(p):
                email = p
            elif re.search(r"\d", p):
                phone = p
            else:
                name = p

        if not is_valid_email(email):
            continue

        email_lower = email.casefold()
        if email_lower in seen_emails:
            continue
        seen_emails.add(email_lower)


        phone = normalize_phone(phone)
        cleaned_rows.append({"name": name, "email": email.strip(), "phone": phone})

    #print(cleaned_rows) used for testing


    with output_file.open("w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["name", "email", "phone"])
        writer.writeheader()
        writer.writerows(cleaned_rows)


if __name__ == "__main__":
    clean_contacts()

## Q2 (1.5 pts) — Unit testing with `unittest`
Create tests in `test_crm_cleanup.py` that cover at least:

1. **Email validation**: valid/invalid variations.  
2. **Phone normalization**: parentheses, dashes, spaces, country code; too-short cases.  
3. **Parsing**: from a small multi-line string (not from a file), assert the exact structured rows (name/email/phone).  
4. **De-duplication**: demonstrate that a case-variant duplicate email is dropped (first occurrence kept).




In [27]:
# Write your answer here
import pytest


@pytest.mark.parametrize("email,expected", [
    ("alice@example.com", True),
    ("bob_123@domain.org", True),
    ("bad-email@", False),
    ("noatsymbol.com", False),
    ("invalid@@example.com", False),
    ("space in@email.com", False),
])
def test_is_valid_email(email, expected):
    assert is_valid_email(email) == expected


@pytest.mark.parametrize("raw,expected", [
    ("+1 (469) 555-1234", "4695551234"),
    ("972-555-9999", "9725559999"),
    ("214 555 8888", "2145558888"),
    ("(972)7772121", "9727772121"),
    ("972.555.77", ""),
    ("abc", ""),
    ("123456789012345", "4567890123"),
])
def test_normalize_phone(raw, expected):
    assert normalize_phone(raw) == expected


def test_parse_contacts_from_string_and_dedup():
    raw_data = """Alice Johnson <alice@example.com> , +1 (469) 555-1234
    Bob Roberts <bob[at]example.com> , 972-555-777
    "Mehdi A." <mehdi.ay@example.org> , (469)555-9999
    Delaram <delaram@example.io>, +1-972-777-2121
    Nima <NIMA@example.io> , 972.777.2121
    duplicate <Alice@Example.com> , 469 555 1234
    """
    rows = parse_contacts_from_string(raw_data)

    expected = [
        {"name": "Alice Johnson", "email": "alice@example.com", "phone": "4695551234"},
        {"name": "Mehdi A.", "email": "mehdi.ay@example.org", "phone": "4695559999"},
        {"name": "Delaram", "email": "delaram@example.io", "phone": "9727772121"},
        {"name": "Nima", "email": "NIMA@example.io", "phone": "9727772121"},
    ]

    assert rows == expected
    # De-duplication check: case-insensitive duplicate removed
    emails = [r["email"].casefold() for r in rows]
    assert len(emails) == len(set(emails))

## Grading rubric (total 3 pts)
- **Q1 (1.5 pts)**  
  - (0.4) File I/O with `pathlib` + graceful `FileNotFoundError` handling  
  - (0.5) Email validation (regex + strip + full match) and filtering  
  - (0.4) Phone normalization and **case-insensitive** de-duplication (keep first)  
  - (0.2) Code clarity (names, minimal docstrings/comments)
- **Q2 (1.5 pts)**  
  - (0.6) Meaningful coverage for email/phone functions (valid & invalid)  
  - (0.6) Parsing & de-dup tests that assert exact expected rows  
  - (0.3) Standard `unittest` structure and readable test names
